In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
from copy import deepcopy

import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets as D
from torchvision import transforms as T

from utils import train_step, test_step
from utils import simulate_scheduler
from utils import BaselineModel
from utils import get_cifar10_dataset, make_dataloader
from utils import check_install_module
check_install_module("plotly")
check_install_module("jupyterplot")

from plotly import express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from jupyterplot import ProgressPlot

# Learning rate를 실시간으로 조정

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 30
batch_size = 64
learning_rate = 0.01
momentum = 0.9
phases = ['train', 'test']

In [ ]:
cifar10_dataset = get_cifar10_dataset()
loader = make_dataloader(cifar10_dataset, batch_size)

In [ ]:
base_cnn = BaselineModel()
nets = [
    BaselineModel().to(device)
    for _ in range(2)
]
_ = [net.load_state_dict(deepcopy(base_cnn.state_dict())) for net in nets]

optimizers = [
    torch.optim.SGD(net.parameters(), learning_rate, 0.9)
    for net in nets
]
criterion = nn.CrossEntropyLoss()

In [ ]:
gamma = 0.9
data = {
    'Learning rate': simulate_scheduler(gamma, num_epochs),
    'Epoch': list(range(num_epochs))
}

px.line(data, x='Epoch', y='Learning rate', width=600, height=400)

In [ ]:
optimizers = [
    torch.optim.SGD(net.parameters(), learning_rate, 0.85)
    for net in nets
]
sched = torch.optim.lr_scheduler.ExponentialLR(optimizers[1], gamma)
conditions = ['without_scheduler', 'with_scheduler']
pp = ProgressPlot(
    plot_names=phases,
    line_names=conditions,
    x_lim=[0, num_epochs*len(loader['train'])],
    x_label='Iteration',
    y_lim=[[0, 3], [50, 100]]
)
logs = [dict(loss=[], accuracy=[]) for _ in conditions]

accs = [0, 0]
for epoch in range(num_epochs):
    for inputs, target in loader['train']:
        losses = [
            train_step(net, inputs, target, optimizer, criterion, device)
            for net, optimizer in zip(nets, optimizers)
        ]
        pp.update([losses, accs])
        [log['loss'].append(loss) for log, loss in zip(logs, losses)]
    
    corrects = [0, 0]
    for inputs, target in loader['test']:
        outputs = [
            test_step(net, inputs, target, device=device)[0]
            for net in nets
        ]
        corrects = [
            (correct + (output.argmax(1).cpu() == target).sum()).item()
            for correct, output in zip(corrects, outputs)
        ]
        
    accs = [
        correct / len(cifar10_dataset['test']) * 100
        for correct in corrects
    ]
    [log['accuracy'].append(acc) for log, acc in zip(logs, accs)]
    
    print(f'Epoch: {epoch+1} accuracy ', end='')
    lrs = [learning_rate, sched.get_lr()[0]]
    for cond, acc, lr in zip(conditions, accs, lrs):
        print(f'{cond}: lr {lr:.5f}-{acc:.2f}', end=' ')
    print()
    sched.step()
pp.finalize()

In [ ]:
loss_df = {lr: log['loss'] for lr, log in zip(conditions, logs)}
acc_df = {lr: log['accuracy'] for lr, log in zip(conditions, logs)}

iteration = list(range(len(loss_df[conditions[0]])))
ext_epoch = [iep+len(loader['train']) for iep in range(0, len(iteration), len(loader['train']))]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_traces(
    [go.Scatter(x=iteration, y=loss_df[lr], name=lr) for lr in conditions],
    secondary_ys=[False for _ in conditions]
)

fig.add_traces(
    [go.Scatter(x=ext_epoch, y=acc_df[lr], name=lr) for lr in conditions],
    secondary_ys=[True for _ in conditions]
)

fig.update_xaxes(title_text='Iteration')
fig.update_yaxes(title_text='Loss', secondary_y=False, range=[0, 3])
fig.update_yaxes(title_text='Accuracy', secondary_y=True, range=[0,100])

fig.show()